In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
from shapely.geometry import LineString
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [5]:
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')

df = gpd.read_file('testing/data/linear-events-3.geojson').set_index('index')

In [6]:
df.lr.add_chaining(inplace=True)
df.lr.active_lrs

LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, closed=left_mod)

In [7]:
dissolved, relation = df.lr.dissolve(return_relation=True)

In [10]:
df.lr.add_geom_m(inplace=True)

In [11]:
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, closed=left_mod)
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry_m, closed=left_mod)

In [7]:
relation.linemerge_m(data=df.geometry)

AttributeError: 'LineString' object has no attribute 'm'